In [1]:
import sys

sys.path.append('..')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import ProcessedDataset, split_data
from tqdm import trange

torch.manual_seed(0)

limit = 10000
num_moves = 40
learning_rate = 1e-3
batch_size = 64
epochs = 20

In [2]:
class Dense(nn.Module):
  def __init__(self, input_dim: int, hidden: list[int]):
    super().__init__()
    layers = []
    for h in hidden:
      layers.append(nn.Linear(input_dim, h))
      layers.append(nn.ReLU())
      input_dim = h
    layers.append(nn.Linear(input_dim, 4))
    self.layers = nn.Sequential(*layers)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = x.permute(0, 2, 1, 3, 4)
    x = x.reshape(x.size(0), -1)
    return self.layers(x)


class Conv(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv3d(6, 64, kernel_size=(5, 3, 3), stride=(2, 1, 1))
    self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 1, 1))

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = x.permute(0, 2, 1, 3, 4)
    x = F.relu(self.conv1(x))
    return F.relu(self.conv2(x))

In [3]:
class Dense1(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense = Dense(15360, [512])

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.dense(x)


class Dense3(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense = Dense(15360, [512, 512, 64])

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.dense(x)


class Dense6(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense = Dense(15360, [2048, 2048, 512, 512, 64])

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.dense(x)


class Conv1(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = Conv()
    self.dense = Dense(16384, [512])

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.conv(x)
    return self.dense(x)


class Conv3(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = Conv()
    self.dense = Dense(16384, [512, 512, 64])

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.conv(x)
    return self.dense(x)


class Conv6(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = Conv()
    self.dense = Dense(16384, [2048, 2048, 512, 512, 64])

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.conv(x)
    return self.dense(x)

In [4]:
def train(model: nn.Module, loader: DataLoader, optimizer: optim.Optimizer) -> float:
  model.train()
  losses = 0
  for moves, evals, times, game_labels in loader:
    optimizer.zero_grad()
    output = model(moves)
    loss = F.cross_entropy(output, game_labels)
    loss.backward()
    optimizer.step()
    losses += loss.item()
  return losses / len(loader)


def evaluate(model: nn.Module, loader: DataLoader) -> float:
  model.eval()
  total, correct = 0, 0
  with torch.no_grad():
    for moves, evals, times, game_labels in loader:
      output = model(moves)
      correct += (output.argmax(dim=1) == game_labels).sum().item()
      total += len(game_labels)
  return correct / total

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = ProcessedDataset(limit, num_moves, 6, device)
train_loader, val_loader, test_loader = split_data(dataset, batch_size)

In [6]:
results = {}

for model in [Dense1(), Dense3(), Dense6(), Conv1(), Conv3(), Conv6()]:
  name = model.__class__.__name__
  print(name)

  model = model.to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  losses, accuracies = [], []
  for epoch in trange(epochs):
    loss = train(model, train_loader, optimizer)
    accuracy = evaluate(model, val_loader)
    losses.append(loss)
    accuracies.append(accuracy)

  results[name] = (losses, accuracies)
  accuracy = evaluate(model, test_loader)
  print(f'Test accuracy: {accuracy:.4f}\n')

Dense1


100%|██████████| 20/20 [01:35<00:00,  4.75s/it]


Test accuracy: 0.7987

Dense3


100%|██████████| 20/20 [01:44<00:00,  5.21s/it]


Test accuracy: 0.7953

Dense6


...

Test accuracy: 0.7865

Conv3


100%|██████████| 20/20 [02:08<00:00,  6.44s/it]


Test accuracy: 0.7845

Conv6


100%|██████████| 20/20 [03:05<00:00,  9.29s/it]


Test accuracy: 0.7412

